# perfSONAR testpoint installation and initial configuration
<p>the following steps will install the `perfsonar-testpoint` bundle</p>
<p>depending on the goals for this node, additional tasks include:
<ul>
<li>configuring this perfSONAR (pS) node to participate in a mesh</li>
<li>setting up this node to participate in GridFTP disk-to-disk testing</p>
</ul>

## pull in the perfsonar-testpoint notebook
<p>using your terminal app log into your fionette as root, and cd into /root</p>
<p>use wget to transfer the perfsonar-testpoint notebook<pre>
wget http://fiona-10-01:8000/perfsonar-testpoint.ipynb -O perfsonar-testpoint.ipynb
</pre>
<p>launch jupyter lab</p>
<pre>jupyter lab --allow-root --ip=`your.fionette.ip`</pre>

## first we will prepare the system for installing the `perfsonar-testpoint` bundle

In [1]:
uname -r

3.10.0-693.17.1.el7.x86_64


In [2]:
sed -i 's/SELINUX=enforcing/SELINUX=disabled/g' /etc/selinux/config
setenforce 0

setenforce: SELinux is disabled


: 1

## setup NTP, bind-utils, traceroute (if not already present)

In [3]:
yum -y install ntp bind-utils traceroute

Loaded plugins: fastestmirror
base                                                     | 3.6 kB     00:00     
epel                                                     | 4.7 kB     00:00     
extras                                                   | 3.4 kB     00:00     
globus                                                   | 2.9 kB     00:00     
perfSONAR                                                | 2.9 kB     00:00     
perfsonar                                                | 2.9 kB     00:00     
updates                                                  | 3.4 kB     00:00     
Loading mirror speeds from cached hostfile
 * perfSONAR: linux.mirrors.es.net
Package ntp-4.2.6p5-25.el7.centos.2.x86_64 already installed and latest version
Resolving Dependencies
--> Running transaction check
---> Package bind-utils.x86_64 32:9.9.4-51.el7_4.2 will be installed
--> Processing Dependency: bind-libs = 32:9.9.4-51.el7_4.2 for package: 32:bind-utils-9.9.4-51.el7_4.2.x86_64
--> Processin

In [4]:
systemctl enable ntpd

# adjust `/etc/ntp.conf` for the workshop NTP server
<p>review the contents of the file</p>

In [5]:
cat /etc/ntp.conf

# For more information about this file, see the man pages
# ntp.conf(5), ntp_acc(5), ntp_auth(5), ntp_clock(5), ntp_misc(5), ntp_mon(5).

driftfile /var/lib/ntp/drift

# Permit time synchronization with our time source, but do not
# permit the source to query or modify the service on this system.
restrict default nomodify notrap nopeer noquery

# Permit all access over the loopback interface.  This could
# be tightened as well, but to do so would effect some of
# the administrative functions.
restrict 127.0.0.1 
restrict ::1

# Hosts on local network are less restricted.
#restrict 192.168.1.0 mask 255.255.255.0 nomodify notrap

# Use public servers from the pool.ntp.org project.
# Please consider joining the pool (http://www.pool.ntp.org/join.html).
server 0.centos.pool.ntp.org iburst
server 1.centos.pool.ntp.org iburst
server 2.centos.pool.ntp.org iburst
server 3.centos.pool.ntp.org iburst

#broadcast 192.168.1.255 autokey	# broadcast server
#broadcastclient			# broadcast client
#broa

<p>using your favorite editor, comment out NTP servers outside of the workshop and add an entry for the local NTP server</p>
<pre>
#server time.apple.com
server 137.164.48.196 iburst
</pre>

## verify the changes

In [6]:
cat /etc/ntp.conf

# For more information about this file, see the man pages
# ntp.conf(5), ntp_acc(5), ntp_auth(5), ntp_clock(5), ntp_misc(5), ntp_mon(5).

driftfile /var/lib/ntp/drift

# Permit time synchronization with our time source, but do not
# permit the source to query or modify the service on this system.
restrict default nomodify notrap nopeer noquery

# Permit all access over the loopback interface.  This could
# be tightened as well, but to do so would effect some of
# the administrative functions.
restrict 127.0.0.1 
restrict ::1

# Hosts on local network are less restricted.
#restrict 192.168.1.0 mask 255.255.255.0 nomodify notrap

# Use public servers from the pool.ntp.org project.
# Please consider joining the pool (http://www.pool.ntp.org/join.html).
server 0.centos.pool.ntp.org iburst
server 1.centos.pool.ntp.org iburst
server 2.centos.pool.ntp.org iburst
server 3.centos.pool.ntp.org iburst

#broadcast 192.168.1.255 autokey	# broadcast server
#broadcastclient			# broadcast client
#broa

In [7]:
systemctl start ntpd

In [8]:
systemctl status ntpd -l

● ntpd.service - Network Time Service
   Loaded: loaded (/usr/lib/systemd/system/ntpd.service; enabled; vendor preset: disabled)
   Active: active (running) since Sat 2018-03-03 21:20:51 PST; 1s ago
  Process: 14621 ExecStart=/usr/sbin/ntpd -u ntp:ntp $OPTIONS (code=exited, status=0/SUCCESS)
 Main PID: 14622 (ntpd)
   CGroup: /system.slice/ntpd.service
           └─14622 /usr/sbin/ntpd -u ntp:ntp -g

Mar 03 21:20:51 gigiot-12 ntpd[14622]: Listen and drop on 1 v6wildcard :: UDP 123
Mar 03 21:20:51 gigiot-12 ntpd[14622]: Listen normally on 2 lo 127.0.0.1 UDP 123
Mar 03 21:20:51 gigiot-12 ntpd[14622]: Listen normally on 3 enp3s0 137.164.48.212 UDP 123
Mar 03 21:20:51 gigiot-12 ntpd[14622]: Listen normally on 4 lo ::1 UDP 123
Mar 03 21:20:51 gigiot-12 ntpd[14622]: Listen normally on 5 enp3s0 fe80::428d:5cff:fef9:678c UDP 123
Mar 03 21:20:51 gigiot-12 ntpd[14622]: Listening on routing socket on fd #22 for interface updates
Mar 03 21:20:51 gigiot-12 systemd[1]: Started Network Time Service.


In [9]:
ntpq -p -c rv

     remote           refid      st t when poll reach   delay   offset  jitter
 chl.la          216.218.254.202  2 u    -   64    1   12.338    3.282   0.065
 doomshack.org   216.218.254.202  2 u    3   64    1   12.361    2.688   0.000
 resolver1.skyfi 216.218.192.202  2 u    -   64    1   29.711   -0.166   0.380
 PBX.cytranet.ne 198.60.22.240    3 u    1   64    1   12.314   -3.299   0.000
 fiona-10-01     69.89.207.99     2 u    -   64    1    0.231   -0.042   0.000
associd=0 status=c011 leap_alarm, sync_unspec, 1 event, freq_not_set,
version="ntpd 4.2.6p5@1.2349-o Wed Apr 12 21:24:06 UTC 2017 (1)",
processor="x86_64", system="Linux/3.10.0-693.17.1.el7.x86_64", leap=11,
stratum=16, precision=-23, rootdelay=0.000, rootdisp=0.075, refid=INIT,
reftime=00000000.00000000  Sun, Dec 31 1899 16:00:00.000,
clock=de4600b8.6f7cce7e  Sat, Mar  3 2018 21:20:56.435, peer=0, tc=3,
mintc=3, offset=0.000, frequency=0.000, sys_jitter=0.000,
clk_jitter=0.000, clk_wander=0.000


## determine default interface

In [10]:
default_iface=$(awk '$2 == 00000000 { print $1 }' /proc/net/route)
echo $default_iface

enp3s0


## confirm `hostname -f` responds with FQHN

In [11]:
hostname -f

gigiot-12.conf.cenic.org


## use hostnamectl to set if its not a valid FQHN

In [ ]:
myip=$(ip addr show dev $default_iface | grep 'inet '| awk '{print $2}' | awk -F "/" '{print $1}')
dig -x $myip +short

In [ ]:
hostnamectl set-hostname `dig -x $myip +short`
hostname -f

## view current link settings on default interface

In [12]:
ip link show dev $default_iface
tc qdisc show

2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq state UP mode DEFAULT qlen 1000
    link/ether 40:8d:5c:f9:67:8c brd ff:ff:ff:ff:ff:ff
qdisc noqueue 0: dev lo root refcnt 2 
qdisc fq 0: dev enp3s0 root refcnt 2 limit 10000p flow_limit 100p buckets 1024 quantum 3028 initial_quantum 15140 


## create a custom udev rules file for tuning the NIC configuration 

In [13]:
if [ ! -f /etc/udev/rules.d/80-prp-fiona-ws--ps-netconf.rules ]; then
cat > /etc/udev/rules.d/80-prp-fiona-ws--ps-netconf.rules <<EOL
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k mtu 9000"
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k txqueuelen 10000"
EOL
chmod 644 /etc/udev/rules.d/80-prp-fiona-ws--ps-netconf.rules
fi

## Use udevadm to reload the rules and trigger the action

In [14]:
/sbin/udevadm control --reload-rules
/sbin/udevadm trigger

## confirm new NIC tuning settings are present

In [15]:
ip link show dev $default_iface
tc qdisc show

2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq state UP mode DEFAULT qlen 10000
    link/ether 40:8d:5c:f9:67:8c brd ff:ff:ff:ff:ff:ff
qdisc noqueue 0: dev lo root refcnt 2 
qdisc fq 0: dev enp3s0 root refcnt 2 limit 10000p flow_limit 100p buckets 1024 quantum 3028 initial_quantum 15140 


## Create a custom sysctl file with desired settings

In [16]:
if [ ! -f /etc/sysctl.d/80-prp-fiona-ws--ps-sysctl.conf ]; then
cat>/etc/sysctl.d/80-prp-fiona-ws--ps-sysctl.conf <<EOL
net.core.rmem_max = 67108864 
net.core.wmem_max = 67108864 
net.ipv4.tcp_rmem = 4096 87380 33554432
net.ipv4.tcp_wmem = 4096 65536 33554432
net.ipv4.tcp_congestion_control=htcp
EOL
fi
chmod 644 /etc/sysctl.d/80-prp-fiona-ws--ps-sysctl.conf

## Adjust kernel boot-time defaults and rebuild grub.cfg

In [17]:
if ! grep --quiet default_hugepagesz /etc/default/grub; then
	sed -i 's/^GRUB_CMDLINE_LINUX=\"[^\"]*/& default_hugepagesz=1GB hugepagesz=1GB hugepages=8 elevator=noop processor.max_cstate=1 intel_iommu=off idle=poll intel_idle.max_cstate=0 nosoftlockup mce=ignore_mce/' /etc/default/grub
	grub2-mkconfig -o /boot/grub2/grub.cfg
fi

In [18]:
cat /etc/default/grub

GRUB_TIMEOUT=5
GRUB_DISTRIBUTOR="$(sed 's, release .*$,,g' /etc/system-release)"
GRUB_DEFAULT=saved
GRUB_DISABLE_SUBMENU=true
GRUB_TERMINAL_OUTPUT="console"
GRUB_CMDLINE_LINUX="crashkernel=auto rhgb quiet default_hugepagesz=1GB hugepagesz=1GB hugepages=8 elevator=noop processor.max_cstate=1 intel_iommu=off idle=poll intel_idle.max_cstate=0 nosoftlockup mce=ignore_mce"
GRUB_DISABLE_RECOVERY="true"


# SAVE your work NOW

In [ ]:
reboot

## confirm intended tuning settings are present 

In [ ]:
ip link show | grep UP | grep -v lo
tc qdisc show

## stop & disable a few un-wanted services and start & enable the `tuned` service

In [19]:
systemctl stop NetworkManager 
systemctl disable NetworkManager
systemctl stop irqbalance.service
systemctl disable irqbalance.service
systemctl stop wpa_supplicant
systemctl disable wpa_supplicant
systemctl start tuned
systemctl enable tuned


## check the current clock rate of the CPU cores

In [20]:
cat /proc/cpuinfo | grep Hz

model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000


## use `tuned-adm` to show the current active profile

In [ ]:
tuned-adm active

## set `network-throughput` at the active profile

In [ ]:
tuned-adm profile network-throughput

## re-check the clock rate of the CPU cores

In [ ]:
cat /proc/cpuinfo | grep Hz

## Install the perfSONAR testpoint bundle (perfsonar-testpoint) and additional tools

<p>The assumption is that CentOS 7 is installed on your machine and that you have access to the necessary tools and configuration to install the perfsonar-testpoint software bundle.</p>
<p>First we will install some addditional and supplementary tools, if not already present on the system</p>

## Installing the perfSONAR testpoint bundle
<p>The instructions below are based on the perfSONAR CentOS Bundle Installation page, here:</p>
<p>http://docs.perfsonar.net/install_centos.html</p>
<p>with modifications for this workshop</p>
## First we need to install the Extra Packages for Enterprise Linux (EPEL)

## point yum to local perfSONAR repo 

In [21]:
cat << EOF > /etc/yum.repos.d/perfsonar.repo
[perfsonar]
name=Extra Packages for Enterprise Linux 7 - $basearch
baseurl=http://137.164.48.196/centos-7-x86_64/perfsonar
failovermethod=priority
enabled=1
gpgcheck=0
EOF

## Refresh the yum cache

In [22]:
yum clean all

Loaded plugins: fastestmirror
Cleaning repos: base epel extras globus perfSONAR perfsonar updates zfs
Cleaning up everything
Maybe you want: rm -rf /var/cache/yum, to also free up space taken by orphaned data from disabled or removed repos
Cleaning up list of fastest mirrors


## Now we can proceed to install the `perfsonar-testpoint` bundle:
<p>NOTE: due to the extensive output from installing this bundle, the call to yum includes the `-q` switch to set quiet mode.  output to `stderr` and any non-zero exit code will appear in the cell below.</p>
<p>NOTE: in a terminal session, run <pre>tail -f /var/log/yum.log</pre> to track the bundle installation</p>

In [23]:
yum -y -q install perfsonar-testpoint

Created symlink from /etc/systemd/system/multi-user.target.wants/owamp-server.service to /usr/lib/systemd/system/owamp-server.service.
Created symlink from /etc/systemd/system/multi-user.target.wants/bwctl-server.service to /usr/lib/systemd/system/bwctl-server.service.
Error unpacking rpm package python-urllib3-1.10.2-3.el7.noarch
error: unpacking of archive failed on file /usr/lib/python2.7/site-packages/urllib3/packages/ssl_match_hostname: cpio: rename failed - Is a directory
error: python-urllib3-1.10.2-3.el7.noarch: install failed
Error unpacking rpm package python-subprocess32-3.2.7-1.el7.centos.x86_64
error: unpacking of archive failed on file /usr/lib64/python2.7/site-packages/subprocess32-3.2.7-py2.7.egg-info: cpio: rename failed - Is a directory
error: python-subprocess32-3.2.7-1.el7.centos.x86_64: install failed
Created symlink from /etc/systemd/system/multi-user.target.wants/perfsonar-meshconfig-agent.service to /usr/lib/systemd/system/perfsonar-meshconfig-agent.service.
Cre

: 1

## install optional `perfsonar-testpoint` packages

In [ ]:
yum -y install perfsonar-toolkit-ntp perfsonar-toolkit-security perfsonar-toolkit-sysctl perfsonar-toolkit-servicewatcher perfsonar-toolkit-systemenv-testpoint

## reload the firewall ruleset

In [ ]:
firewall-cmd --reload

## check current active firewall rules

In [ ]:
firewall-cmd --list-all

## NOTE: if the output of entry for `services:` entry does include `bwctl` or `owamp-control` re-run the script to configure the firewall rules for perfSONAR

In [ ]:
/usr/lib/perfsonar/scripts/configure_firewall install

## re-check firewall rules

In [ ]:
firewall-cmd --list-all

## use `systemctl` to check the status of processes

In [ ]:
systemctl status postgresql-9.5 -l
systemctl status httpd -l
systemctl status pscheduler* -l
systemctl status perfsonar-meshconfig-agent -l

## Save your work NOW
## Reboot the server and verify everything came up as expected
## You will need to re-login to your machine and launch the notebook

In [ ]:
reboot

In [ ]:
systemctl status postgresql-9.5 -l
systemctl status httpd -l
systemctl status pscheduler* -l
systemctl status perfsonar-meshconfig-agent -l

## update the `meshconfig-agent` to add a reference for the group mesh json
<p>review the current contents</p>

In [ ]:
cat /etc/perfsonar/meshconfig-agent.conf

## update the `meshconfig-agent.conf` with a mesh stanza referencing the group mesh json

In [ ]:
cd /etc/perfsonar
wget http://fiona-10-01:8000/host-to-group -O /usr/local/etc/prp-fiona-ws--host-to-group
mygrp=$(grep $(hostname -s) /usr/local/etc/prp-fiona-ws--host-to-group | awk '{print $4}')
if [ ! -f /etc/perfsonar/meshconfig-agent.conf.orig ]; then
cp /etc/perfsonar/meshconfig-agent.conf /etc/perfsonar/meshconfig-agent.conf.orig
fi
cat >> /etc/perfsonar/meshconfig-agent.conf <<EOL
<mesh>
   configuration_url http://gigiot-ma-$mygrp/prp-fiona-ws-mesh--gigiot-ma-$mygrp.json
   #validate_certificate         0
   #ca_certificate_file          /etc/pki/tls/certs/ca-bundle.crt
   configure_archives 1
</mesh>
EOL
chown perfsonar:perfsonar /etc/perfsonar/meshconfig-agent.conf
chmod 644 /etc/perfsonar/meshconfig-agent.conf

In [ ]:
cat /etc/perfsonar/meshconfig-agent.conf

## restart the `perfsonar-meshconfig-agent` process

In [ ]:
systemctl restart perfsonar-meshconfig-agent

## verify the contents of `meshconfig-agent-tasks.conf` to determine if the mesh json has been processed correctly
<p>it should have `<test>` stanzas which includes sections for test schedule, parameters, targets, and measurement archives</p> 

In [ ]:
cat /etc/perfsonar/meshconfig-agent-tasks.conf

# confirming a succesful installation and mesh participation


## open a terminal session and run the following to help determine the functionality of the system
<ul>
<li> `pscheduler monitor` to show an updating display of completed, running, and pending tasks</li>
<li> `pscheduler task throughput --dest some.other.ps.node` to run a throughput (iperf3) test</li>
<li> `pscheduler task latency --dest some.other.ps.node` to run a latency/loss (owping) test</li> 
</ul>

# perfSONAR testpoint appendix: perfSONAR testpoint installation troubleshooting 

## pscheduler processes fail to start 
<p>there are reports of a recent dependency breakage in the perfSONAR repo which may cause `httpd` to be absent from the bundle installation. pScheduler will not function without httpd.  if you attempt to use pscheduler and you receive an error message that says:<pre>Unable to find pScheduler on localhost</pre> this can be an indication that httpd is missing.</p>
<p>install and enable httpd if not present on the system</p>

In [ ]:
yum -y -q install httpd
systemctl enable httpd
systemctl start httpd
systemctl status httpd -l

## python library incompatibility on systems having both perSONAR and GridFTP
<p>We have seen an issue with the GridFTP installation where an incompatible python library is installed and this causes the pscheduler processes to not start. If your pscheduler-* processes do not start you can try the following:</p>

In [ ]:
pip uninstall urllib3
yum install -y python-urllib3
systemctl start pscheduler-*
systemctl status pscheduler-* -l

# Setting up scheduled GridFTP testing
<p>the following steps will set up cron jobs to accomplish</p>
<ul>
<li>a schedule of GridFTP transfers from a group of participating nodes</li>
<li>parsing the GridFTP transfer log and uploading the results to a central Measurement Archive (MA)</li>
</ul>

## wget the host-to-group map and the group-specific script for GridFTP and make it executable

In [ ]:
wget http://fiona-10-01:8000/host-to-group -O /usr/local/etc/prp-fiona-ws--host-to-group
mygrp=$(grep $(hostname -s) /usr/local/etc/prp-fiona-ws--host-to-group | awk '{print $4}')
wget http://fiona-10-01:8000/cron-load-gridftp-grp-0$mygrp.sh -O /usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh
chmod 755 /usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh

## inspect the contents of the script

In [ ]:
cat /usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh

## using your favorite editor, update the `GRIDFTP_API_KEY` entry
<p>update the <pre>GRIDFTP_API_KEY</pre> entry with the API key generated earlier for the gridftp user on the MA for your group<p>

## Grab a helper script that gragefully times out the globus url copy if it exceeds a value in seconds.

In [ ]:
mkdir /opt/esmond-gridftp/
wget http://fiona-10-01:8000/timeout.sh -O /usr/local/bin/timeout.sh
chmod 755 /usr/local/bin/timeout.sh

## This script scrapes the gridftp transfer log for transfers that it has not yet processed
<p>The results are uploaded to the central measurement archive running on the MA host for your group.</p>

In [ ]:
/usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh 


## Running this file the first time pulls in the cron-gridftp-transfer-mesh for this host

In [ ]:
chmod 644 /etc/cron.d/cron-gridftp-transfer-mesh-grp-0$mygrp

## Inspect the cron job

In [ ]:
cat /etc/cron.d/cron-gridftp-transfer-mesh-grp-0$mygrp